# **과제2 : 챗봇 만들기2**

## 0.미션

* 예비 에이블러들을 위한 QA 챗봇 모델 만들기2
    * Vector DB에 데이터 추가하기
    * Retriever, memory, LLM를 연결하기
    * 실행시 이력 DB 생성하고 기록하기
    * test

## **1.환경준비**

### (1) 라이브러리 Import

In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
from datetime import datetime

import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (2) OpenAI API Key 확인
* 환경변수로 등록된 Key 확인하기

In [2]:
# 환경변수에서 키 불러오기
api_key = os.getenv('OPENAI_API_KEY')
print(api_key[:5]) 

sk-pr


* 만약 환경변수 키 설정이 잘 안된다면 아래 코드셀의 주석을 해제하고, 자신의 api key를 입력하고 실행
    * 아래 코드는 키 지정을 **임시**로 수행함.
    * 파이썬 파일(.ipynb, .py)안에서 매번 수행해야 함.

In [62]:
# os.environ['OPENAI_API_KEY'] = '여러분의 OpenAI API키' 
# openai.api_key = os.getenv('OPENAI_API_KEY')

## **2.Vector DB 만들기**

* 데이터 로딩 
    * 1일차에서 제공한 csv 파일의 구조를 그대로 이용
    * 에이블스쿨 홈페이지 FAQ 데이터 수집(https://aivle.kt.co.kr/home/brd/faq/main?mcd=MC00000056)
        * 모든 질문을 csv 형태로 저장
    * 데이터프레임으로 저장하기

In [3]:
df = pd.read_csv('aivleschool_qa.csv')
df.head()

,구분,QA
0,모집/선발,최종 학력 또는 전공과 관계없이 지원할 수 있나요?\nKT 에이블스쿨은 정규 4년제...
1,모집/선발,35세 이상은 지원할 수 없나요?\n본 교육 과정은 34세 이하를 대상으로 하는 교...
2,모집/선발,미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다.\n1) 기간의 정함...
3,모집/선발,"직장인도 지원할 수 있나요?\nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시..."
4,모집/선발,아르바이트를 하고 있는데 지원할 수 있나요?\n고용보험에 가입이 되어 있는 경우 1...


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# 웹드라이버 설정 (경로는 본인의 webdriver 경로로 변경)
driver = webdriver.Chrome() 

# 웹 페이지 열기
url = 'https://aivle.kt.co.kr/home/brd/faq/main?mcd=MC00000056'
driver.get(url)

SCROLL_PAUSE_TIME = 1.5
last_height = driver.execute_script("return document.body.scrollHeight")

category = []
qa = []

try:
    for i in range(10):
        more_button = driver.find_element(By.CSS_SELECTOR, 'a.btn.gn')
        more_button.click()
        time.sleep(1)
except:
    print('더보기 버튼이 없습니다')

# class가 'open'인 모든 li 태그 찾기
li_elements = driver.find_elements(By.CLASS_NAME, 'open')

# 각 li 태그 클릭하여 내용 로드
for li in li_elements:
    li.click()
    time.sleep(1)  # 클릭 후 잠시 대기 (콘텐츠 로드 시간 필요)

# 페이지 소스를 BeautifulSoup으로 파싱
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 모든 div 태그 중 class가 'question'인 태그 찾기
cat_tags = soup.find_all('p', class_='cate')
question_tags = soup.find_all('p', class_='subject')
answer_tags = soup.find_all('div', class_='answer')

print(len(cat_tags), len(question_tags), len(answer_tags))

for cat, question, answer in zip(cat_tags, question_tags, answer_tags):
    category.append(cat.text.strip())
    qa.append(question.text + '\n' + answer.text.strip())

# 브라우저 종료
driver.quit()

# 결과 출력
for a, b in zip(category, qa):
    print(a, b)


더보기 버튼이 없습니다
68 68 68
모집/선발 1) 최종 학력 또는 전공과 관계없이 지원할 수 있나요? 
KT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자를 대상으로 하는 교육입니다. 전공에 관계 없이 명시된 지원 자격에 부합한다면 모두 지원 가능합니다. 다만, AI개발자 Track은 기본적인 코딩역량이 필요합니다.
모집/선발 2) 35세 이상은 지원할 수 없나요?
본 교육 과정은 34세 이하를 대상으로 하는 교육입니다. 단, 모집시점에 35세라도 해당연도 1월 1일 이후 생일자는 지원이 가능합니다. (예: 5기 지원 기준 1989년 1월 1일 이후 출생자)
모집/선발 3) 미취업자의 기준이 뭔가요?
미취업자의 기준은 아래와 같습니다. 
1) 기간의 정함이 있는 근로인 경우,  
2) 고용보험에 미가입한 경우, 
3) 고용보험에 가입되어 있더라도 15시간/주 미만 근로인 경우 
단, 어떠한 경우에도 교육을 풀타임(09:00~18:00)으로 들을 수 있어야 교육 참여가 가능합니다.
모집/선발 4) 직장인도 지원할 수 있나요? 
KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가능합니다.
향후 취업여부를 확인할 예정입니다.
모집/선발 5) 아르바이트를 하고 있는데 지원할 수 있나요? 
고용보험에 가입이 되어 있는 경우 15시간/주 미만 근로인 경우에만 미취업자로 간주하여 지원이 가능합니다. 
고용보험에 가입되어 있지 않는 경우에는 특수형태근로자/고용보험미가입근로자로 분류되어 근로시간과 훈련시간이 중복되지 않는다면 훈련 수강은 가능하나, 훈련장려금이 지급되지 않습니다. 
다만, 특수형태근로자의 경우 해촉증명서, 퇴사 사실확인증명원(사업주, 근로자 직인/서명포함)을 제출하는 경우 훈련장려금 지급이 가능하므로, 
고용형태 관련 세부사항은 거주지 관할 고용센터(HRD-Net 확인) 또는 훈련 권역별 고용센터로 문의하시기 바랍니다. 
※ 수도권: 성남고용센터 
      충남/충북: 대전고용센터 
      대구/경북: 대구고용센

In [6]:
df = pd.DataFrame({'구분' : category, 'QA' : qa})
df.tail()

,구분,QA
63,기타,64) AICE (기존 AIFB) 자격증이 있어야 유리한가요?\n네. AICE (기...
64,기타,65) AICE 시험관련 문의는 어디에 해야하나요? \nhelp@aice.study...
65,기타,66) 국민취업지원제도를 신청하고 싶습니다. \n국민취업지원제도 신청과 관련 사항은...
66,기타,67) 국민취업지원제도를 안해도 교육 참여되나요? \n국민취업제도를 신청하지 않아도...
67,기타,68) 국민취업지원제도 관련 직업훈련탐색표가 필요합니다. \n직업훈련탐색표는 고용센...


In [7]:
df.to_csv('aivleschool_qa.csv', index=False)

In [8]:
test_df = pd.read_csv('aivleschool_qa.csv')
test_df.head()

,구분,QA
0,모집/선발,1) 최종 학력 또는 전공과 관계없이 지원할 수 있나요? \nKT 에이블스쿨은 정규...
1,모집/선발,2) 35세 이상은 지원할 수 없나요?\n본 교육 과정은 34세 이하를 대상으로 하...
2,모집/선발,3) 미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다. \n1) 기간...
3,모집/선발,"4) 직장인도 지원할 수 있나요? \nKT 에이블스쿨은 미취업자를 대상으로 하며, ..."
4,모집/선발,5) 아르바이트를 하고 있는데 지원할 수 있나요? \n고용보험에 가입이 되어 있는 ...


* 벡터 데이터베이스
    * 1일차 벡터 데이터베이스를 그대로 이용
        * Embedding 모델 : text-embedding-ada-002
        * DB 경로 : ./db



* 데이터 입력
    * 기존 입력을 모두 제거하고 추가 사항만 모두 입력
    * meta data로 '구분' 칼럼 값 추가하기

In [88]:
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')

database = Chroma(persist_directory='./db', embedding_function=embeddings)

In [91]:
text_list = df['QA'].to_list()
metadata = []
cat_list = df['구분'].to_list()
for cat in cat_list:
    temp = {}
    temp['구분'] = cat
    metadata.append(temp)
    
documents = [Document(page_content = text_list[i], metadata = metadata[i]) for i in range(len(text_list))]

print(documents)

[Document(page_content='1) 최종 학력 또는 전공과 관계없이 지원할 수 있나요? \nKT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자를 대상으로 하는 교육입니다. 전공에 관계 없이 명시된 지원 자격에 부합한다면 모두 지원 가능합니다.\xa0다만, AI개발자 Track은 기본적인 코딩역량이 필요합니다.', metadata={'구분': '모집/선발'}), Document(page_content='2) 35세 이상은 지원할 수 없나요?\n본 교육 과정은 34세 이하를 대상으로 하는 교육입니다.\xa0단, 모집시점에 35세라도 해당연도 1월 1일 이후 생일자는 지원이 가능합니다. (예: 5기 지원 기준 1989년 1월 1일 이후 출생자)', metadata={'구분': '모집/선발'}), Document(page_content='3) 미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다. \n1) 기간의 정함이 있는 근로인 경우,  \n2) 고용보험에 미가입한 경우, \n3) 고용보험에 가입되어 있더라도 15시간/주 미만 근로인 경우 \n단, 어떠한 경우에도 교육을 풀타임(09:00~18:00)으로 들을 수 있어야 교육 참여가 가능합니다.', metadata={'구분': '모집/선발'}), Document(page_content='4) 직장인도 지원할 수 있나요? \nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가능합니다.\n향후 취업여부를 확인할 예정입니다.', metadata={'구분': '모집/선발'}), Document(page_content='5) 아르바이트를 하고 있는데 지원할 수 있나요? \n고용보험에 가입이 되어 있는 경우 15시간/주 미만 근로인 경우에만 미취업자로 간주하여 지원이 가능합니다. \n고용보험에 가입되어 있지 않는 경우에는 특수형태근로자/고용보험미가입근로자로 분류되어 근로시간과 훈련시간이 중복되지 않는다면 훈련 수강은 가능하나, 훈련장려금이 지급되지 않습니다. \n다만

* 입력된 데이터 조회

In [93]:
database.add_documents(documents)

['f0424648-1039-4789-bf63-777bfb30ef73',
 '90aed9ec-b354-46ad-88e3-421cc7302601',
 '30ba200f-384b-40e5-929b-3f2a10868ba9',
 'd0e4007d-f2f3-49e8-b7c5-bb3d1afcf705',
 '4b24752d-3ffe-4e15-8344-c7a254d98b15',
 '8dd613eb-8e0c-4f85-b335-e8eb184697eb',
 'ae8d8ddd-0dac-4c50-920b-f2d6e08a45d3',
 '102defd5-c4d6-49cc-85f0-4fff5dd75b74',
 '01655d49-372f-43bf-9919-77b7ee106e8f',
 '434b4957-f1cd-49e0-b786-1063c1d64119',
 '83818815-f2df-4e66-811c-8de97dad0bf6',
 '22232f2c-898f-4053-ad0d-551d7520fb19',
 'e9ede0f3-2c34-443c-a699-08bf8054bac6',
 '711465f0-d380-43cc-8840-e2d16c3065a8',
 '1e7f5da3-989d-474c-8000-b513d2b00257',
 'fcc8777e-0026-498e-a299-a4be0bbde4ab',
 '79b0345c-40fd-43ca-a4db-7ac0515239d4',
 '38874758-18d3-49dc-b194-8738ed13b69a',
 'e062cbec-2501-40b2-a3e5-d54a6a709f21',
 'af9ad1d3-260f-41d4-8cf1-4d796ed1df5b',
 'e0386208-bf6c-420b-bb50-fcef920acc46',
 'f1fac759-5cdf-4b1d-855c-464fcb7382ec',
 'd8b38e0f-eba4-4847-9018-3b2b0af5c9ac',
 '370b40cd-8cb0-4b94-b01d-ba7f4364dbc0',
 'dcdceb01-eecf-

In [94]:
database.get()

{'ids': ['012f7027-74ae-4f59-aad5-ab1559ac295c',
  '01655d49-372f-43bf-9919-77b7ee106e8f',
  '05454b67-8a26-436c-9dce-bbb5e8f0902e',
  '065e764b-d368-43d3-8f2e-139c24ef78e3',
  '0aada355-d5b0-4c5c-a0b1-0718c3eb33fa',
  '0edab09e-384b-473d-92fd-c466fc155516',
  '102defd5-c4d6-49cc-85f0-4fff5dd75b74',
  '108482e0-8dd6-489d-a88e-89c7bb17e2e6',
  '19ab1821-ff0a-4800-83cd-0043fc4c50e4',
  '19ec4a0c-940b-47e2-9479-3ca59b3debf5',
  '1e7f5da3-989d-474c-8000-b513d2b00257',
  '22232f2c-898f-4053-ad0d-551d7520fb19',
  '279d2e2d-03cc-4e3d-b6c8-b331316120b5',
  '2db569fb-4129-4b43-998e-215102620cb3',
  '30ba200f-384b-40e5-929b-3f2a10868ba9',
  '31d50512-24d6-42f6-b524-cd2d9d6e8ac1',
  '321d18cd-03f1-4982-ada0-1191a3df7146',
  '370b40cd-8cb0-4b94-b01d-ba7f4364dbc0',
  '37fb1f46-678e-4a70-b040-7ad51c879edc',
  '3808efa3-61da-430f-b758-4611ee4a6b6b',
  '38582224-bcd2-4617-8368-7728975f2c71',
  '38874758-18d3-49dc-b194-8738ed13b69a',
  '38dcc43d-5355-49e3-ac47-6c0c233e720a',
  '3c5344cc-2b66-4125-8ccf-

## **3.RAG+LLM모델**

* 모델 : ConversationalRetrievalChain
    * LLM 모델 : gpt-3.5-turbo
    * retriever : 벡터DB
        * 유사도 높은 문서 3개 가져오도록 설정
    * memory 사용
* 요구사항
    * 질문 history 관리를 위한 이력 저장 DB 생성
        * DB 명 : db_chatlog
        * 테이블 명 : history
            * id INTEGER PRIMARY KEY : 이렇게 설정하면 자동증가 값으로 채워짐
            * datetime TEXT : 질문시점 yyyy-mm-dd hh:mi:ss
            * query TEXT : 질문
            * sim1 REAL : 첫번째 문서의 유사도 점수
            * sim2 REAL : 두번째 문서의 유사도 점수
            * sim3 REAL : 세번째 문서의 유사도 점수
            * answer TEXT : 답변
        * 유사도 점수는 similarity_search_with_score 메서드를 이용해서 저장해야 함
        * 질문과 답변이 진행될 때마다 history 테이블에 데이터 입력

* 관리용 DB, 테이블 생성

In [95]:
path = './db_chatlog/db_chatlog.db'
conn = sqlite3.connect(path)

In [96]:
# 커서 객체 생성
cursor = conn.cursor()

# test 테이블 생성
cursor.execute('''
CREATE TABLE IF NOT EXISTS history (
    id INTEGER PRIMARY KEY, 
    datetime TEXT, 
    query TEXT, 
    sim1 REAL, 
    sim2 REAL, 
    sim3 REAL, 
    answer TEXT
)
''')

# 변경사항 커밋 (저장)
conn.commit()

# 연결 종료
conn.close()

* 모델 선언

In [97]:
embeddings = OpenAIEmbeddings(model = "text-embedding-ada-002")
database = Chroma(persist_directory = "./db", embedding_function = embeddings)
chat = ChatOpenAI(model="gpt-3.5-turbo")

k=3
retriever = database.as_retriever(search_kwargs={"k": k})

# 대화 메모리 생성
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", 
                                  output_key="answer", return_messages=True)

# ConversationalRetrievalQA 체인 생성
qa = ConversationalRetrievalChain.from_llm(llm=chat, retriever=retriever, memory=memory, # memory 지정
                                           return_source_documents=True,  output_key="answer") # output_key, memory의 값과 맞춰준다

* 모델 사용 및 이력 확인

In [98]:
from datetime import datetime

while True:
    # ① 연결
    conn = sqlite3.connect(path)
    
    query = input('질문 > ')
    query = query.strip()
    # 현재 시간
    dt = datetime.now()
    dt = dt.strftime('%Y-%m-%d %H:%M:%S')
    result = database.similarity_search_with_score(query, k = 3)
    scores = []
    for doc in result:
        scores.append(round(doc[1], 5))
        
    print(f'질문 : {query}')
    print('-' * 20)
    if len(query) == 0:
        # ③ 연결 종료
        conn.close()
        break
    result = qa({"question": query})
    print(f'답변 : {result["answer"]}')
    
    # ② 작업 : to_sql
    data = pd.DataFrame({'datetime': [dt], 'query': [query], 'sim1': [scores[0]], 'sim2': [scores[1]], 'sim3': [scores[2]], 'answer':[result["answer"]]})
    data.to_sql('history', conn, if_exists='append', index=False) # test 테이블이 있으면 insert, 없으면 생성

    df = pd.read_sql('SELECT * FROM history', conn)
    display(df)

    
    print('=' * 50)

질문 : 노트북은 언제줘?
--------------------
답변 : 교육기간 동안 노트북을 제공해드립니다. 실제 교육 시작일에 수령하실 수 있습니다.
질문 : 교육 시작일은 언제야?
--------------------
답변 : 교육 시작일은 별도의 공지가 있어야 확인할 수 있습니다. 질문하신 교육의 시작일에 대한 정보는 현재 알 수 없습니다. 궁금하신 경우 해당 교육기관에 문의하시는 것을 권장드립니다.
질문 : 교육 시작일을 알려줘
--------------------
답변 : 교육 시작일은 KT 에이블스쿨에서는 미취업자를 대상으로 하며, 재직자는 지원이 불가능하며, 향후 취업 여부를 확인할 예정이라고 합니다. 정확한 교육 시작일은 문의하시는 KT 에이블스쿨 측에 문의하시는 것이 좋을 것 같습니다.
질문 : 
--------------------


In [100]:
# ① 연결
conn = sqlite3.connect(path)

# ② 작업 : select
df = pd.read_sql('SELECT * FROM history', conn)
display(df)

# ③ 연결 종료
conn.close()

,id,datetime,query,sim1,sim2,sim3,answer
0,1,2024-06-04 16:17:08,노트북은 언제줘?,0.27882,0.31682,0.38283,교육기간 동안 노트북을 제공해드립니다. 실제 교육 시작일에 수령하실 수 있습니다.
1,2,2024-06-04 16:17:24,교육 시작일은 언제야?,0.30967,0.31098,0.31582,교육 시작일은 별도의 공지가 있어야 확인할 수 있습니다. 질문하신 교육의 시작일에 ...
2,3,2024-06-04 16:17:59,교육 시작일을 알려줘,0.32940,0.33647,0.34262,"교육 시작일은 KT 에이블스쿨에서는 미취업자를 대상으로 하며, 재직자는 지원이 불가..."
